# Pedestrian Level of Service

In [109]:
import numpy as np
import pandas as pd
import geopandas as gpd
import psrcelmerpy

In [110]:
# Round to nearest 5
def myround(x, base=5):
    return base * round(x/base)

In [111]:
eg_conn = psrcelmerpy.ElmerGeoConn()

# Load network shapefile
gdf = gpd.read_file(r'R:\e2projects_two\SoundCast\Inputs\dev\networks\2023\network_2023_v3\shapefiles\AM\AM_edges.shp')

# Reformat speed limit and lanes to match the bike stress data
# Creating a new column for speed limits where minimum is capped at 25 and max at 35
gdf['SpeedLimit_new'] = gdf['ul2'].copy()
gdf.loc[gdf['ul2'] <= 20, 'SpeedLimit_new'] = 20
gdf.loc[gdf['ul2'] >= 50, 'SpeedLimit_new'] = 50
gdf['SpeedLimit_new'] = myround(gdf['SpeedLimit_new'].astype('int'))

# Set maximum lanes at 6 and minimum atb 2
gdf['lanes_new'] = gdf['lanes'].copy().astype('int')
gdf.loc[gdf['lanes'] >= 3, 'lanes_new'] = 3
gdf.loc[gdf['lanes'] <1, 'lanes_new'] = 1

In [112]:
gdf['SpeedLimit_new'].unique()

array([20., 25., 40., 35., 30., 50., 45.])

In [113]:
gdf['lanes_new'].unique()

array([2, 1, 3])

In [114]:
df_data = pd.read_csv('wsdot_ped_stress_table.csv')

In [115]:
df_data.head()
df_data['AADT_class'] = df_data['AADT_min'].astype('str')+'-'+df_data['AADT_max'].astype('str')
df_data['AADT_class'].value_counts()

1501-3000      7
3001-999999    7
0-750          7
0-999999       7
7001-999999    7
751-1500       7
0-7000         7
Name: AADT_class, dtype: int64

In [116]:
# Load AADT from network_results
df_network = pd.read_csv(r'L:\RTP_2022\final_runs\sc_rtp_2018_final\soundcast\outputs\network\network_results.csv')

In [117]:
df_tveh = df_network.groupby(['ij']).sum()[['@tveh']].reset_index()

gdf = gdf.merge(df_tveh, left_on='id', right_on='ij', how='left')
gdf.rename(columns={'@tveh': 'AADT'}, inplace=True)

# Some links do not have traffic, so set AADT to 0
gdf['AADT'] = gdf['AADT'].fillna(0)

# Put AADT in classes

# For 1-lane streets
gdf.loc[(gdf['lanes_new'] == 1) & (gdf['AADT']<=750), 'AADT_class'] = '0-750'
gdf.loc[(gdf['lanes_new'] == 1) & (gdf['AADT']>750) & (gdf['AADT']<=1500), 'AADT_class'] = '751-1500'
gdf.loc[(gdf['lanes_new'] == 1) & (gdf['AADT']>1500) & (gdf['AADT']<=3000), 'AADT_class'] = '1501-3000'
gdf.loc[(gdf['lanes_new'] == 1) & (gdf['AADT']>3000), 'AADT_class'] = '3001-999999'

# For 2-lanes streets
gdf.loc[(gdf['lanes_new'] == 2) & (gdf['AADT']<=7000), 'AADT_class'] = '0-7000'
gdf.loc[(gdf['lanes_new'] == 2) & (gdf['AADT']>7000), 'AADT_class'] = '7001-999999'

# For 3+ lane streets
gdf.loc[(gdf['lanes_new'] == 3), 'AADT_class'] = '0-999999'

In [118]:
gdf['AADT_class'].value_counts()

0-750          26622
0-7000         25285
3001-999999    14557
7001-999999     8368
1501-3000       7777
751-1500        4480
0-999999        3710
Name: AADT_class, dtype: int64

In [119]:
len(gdf)

90799

In [120]:
df_data.columns

Index(['Lanes', 'SpeedLimit', 'AADT_min', 'AADT_max', 'PTLS', 'AADT_class'], dtype='object')

In [121]:
gdf = gdf.merge(df_data, left_on=['lanes_new','AADT_class','SpeedLimit_new'], right_on=['Lanes','AADT_class','SpeedLimit'], how='left')

# Create new column names because ArcGIS is a dinosaur that can't handle more than 10 character headers
gdf.rename(columns={'BikeFacility':'bkfac', 'SpeedLimit':'speed', 'AADT_class': 'vol'}, inplace=True)

In [122]:
gdf[gdf['PTLS'].isnull()]

,direction,i,j,length,modes,type,lanes,vdf,ul1,ul2,...,SpeedLimit_new,lanes_new,ij,AADT,vol,Lanes,speed,AADT_min,AADT_max,PTLS


In [123]:
gdf['ij'] = gdf['i'].astype('str')+'-'+gdf['j'].astype('str')
gdf['ji'] = gdf['j'].astype('str')+'-'+gdf['i'].astype('str')

gdf['ji_exists'] = 0
gdf.loc[gdf['ji'].isin(gdf['ij']), 'ji_exists'] = 1

# Get a ji dataframe
gdf_ji = gdf[gdf['ji_exists']==1].copy()
gdf_ji.drop(['ij','geometry'], axis=1, inplace=True)
gdf_ji.rename(columns={'ji': 'ij'}, inplace=True)

# merge that to gdf

gdf_new = gdf.copy()
gdf_new = gdf_new[['ij','upslp',  'bkfac','Lanes', 'speed', 'PTLS','geometry','PSRCEdgeID']].merge(gdf_ji[['ij','upslp',  'bkfac','Lanes', 'speed', 'PTLS']],
                                                                                      how='left', on='ij', suffixes=['_IJ','_JI'])
gdf_new.head()

,ij,upslp_IJ,bkfac_IJ,Lanes_IJ,speed_IJ,PTLS_IJ,geometry,PSRCEdgeID,upslp_JI,bkfac_JI,Lanes_JI,speed_JI,PTLS_JI
0,2505-79004,0.000000,0.0,2,20,2,"LINESTRING (1272486.412 309313.000, 1272586.82...",874.0,0.000000,0.0,2.0,20.0,2.0
1,2505-71782,0.000000,0.0,2,20,2,"LINESTRING (1272486.412 309313.000, 1272379.82...",872.0,0.000000,0.0,2.0,20.0,2.0
2,2505-75120,0.000000,0.0,2,20,2,"LINESTRING (1272486.412 309313.000, 1272491.52...",873.0,0.000000,0.0,2.0,20.0,2.0
3,79117-79004,0.186476,0.0,1,25,1,"LINESTRING (1274401.412 310281.765, 1274386.64...",131369.0,0.000000,0.0,1.0,25.0,1.0
4,79004-72460,0.002948,0.0,1,25,1,"LINESTRING (1274354.882 310296.588, 1274316.11...",131706.0,0.071124,0.0,1.0,25.0,1.0


In [124]:
gdf_new.to_file('2023_ped_stress_index.shp')

In [125]:
len(gdf_new)

90799